# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

607

In [3]:
cities[:5]

['rikitea', 'cairns', 'ust-nera', 'ushuaia', 'bredasdorp']

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).


In [4]:
weather_api_key

'c8b9cf8e041808818c631648e43e7413'

In [5]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

# print(json.dumps(requests.get(query_url + cities[0]).json(), indent=3))

data = requests.get(query_url + cities[0]).json()

In [6]:
data.keys()
# if curly brackets, use .keys to find the keys of interest

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])

In [7]:
data['weather'][0]

{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}

In [9]:
# DF to store all cities and city number (uses citipy)
city_df = pd.DataFrame({'City': cities})
print("Beginning Data Retrieval")
print("----------------------------")


# Sets parameters for calling city name


# Target Url for API calls
query_url = "http://api.openweathermap.org/data/2.5/weather"

# Create add'l fields for later api calls

city_df['Lat'] = ''
city_df['Lng'] = ''
city_df['Max Temp'] = ''
city_df['Humidity'] = ''
city_df['Cloudiness'] = ''
city_df['Wind Speed'] = ''
city_df['Country'] = ''
city_df['Date'] = ''

city_count = 0

# Loop through the API
for index,row in city_df.iterrows():

    city_name = row['City']
#     city_id = row['id']
    
    # Set query parameters
    params = {'q': city_name,'units': 'imperial','appid': weather_api_key,}
    response = requests.get(query_url, params = params).json()
    
    try:
       
        city_df.loc[index, 'Lat'] = response['coord']['lat']
        city_df.loc[index, 'Lng'] = response['coord']['lon']
#         citydf.loc[index, 'Max Temp (F)'] = response['main']['temp_max']
#         citydf.loc[index, 'Humidity (%)'] = response['main']['humidity']
#         citydf.loc[index, 'Cloudiness (%)'] = response['clouds']['all']
#         citydf.loc[index, 'Wind Speed'] = response['wind']['speed']
#         city_df.loc[index, 'Date'] = response['dt']
        
        # Update city number
        city_count += 1
        
        # Print log of each city being processed (only if all data is available)
        print(f'Processing Record {city_count} | {city_name}')
        
    except (KeyError, IndexError):

        print("City not found. Skipping...")


## Every time the below three lines are included an error occurs.  So they are commented out until this is resolved.
# print("----------------------------")
# print("Data Retrieval Complete")
# print("----------------------------")

Beginning Data Retrieval
----------------------------
Processing Record 1 | rikitea
Processing Record 2 | cairns
Processing Record 3 | ust-nera
Processing Record 4 | ushuaia
Processing Record 5 | bredasdorp
Processing Record 6 | qaanaaq
Processing Record 7 | tuktoyaktuk
Processing Record 8 | yaan
Processing Record 9 | cabo san lucas
Processing Record 10 | gera
Processing Record 11 | mahebourg
Processing Record 12 | kamaishi
Processing Record 13 | bandarbeyla
Processing Record 14 | subotica
Processing Record 15 | port moresby
Processing Record 16 | harper
Processing Record 17 | saint-philippe
City not found. Skipping...
Processing Record 18 | vaini
Processing Record 19 | hithadhoo
Processing Record 20 | phonhong
Processing Record 21 | honningsvag
Processing Record 22 | hobart
City not found. Skipping...
Processing Record 23 | langevag
City not found. Skipping...
Processing Record 24 | atuona
Processing Record 25 | jamestown
Processing Record 26 | arlit
Processing Record 27 | dikson
Proc

Processing Record 229 | port augusta
Processing Record 230 | eyl
Processing Record 231 | saint-francois
Processing Record 232 | thinadhoo
Processing Record 233 | longyearbyen
Processing Record 234 | ponta do sol
Processing Record 235 | nikolskoye
Processing Record 236 | mirabad
Processing Record 237 | beringovskiy
City not found. Skipping...
Processing Record 238 | taree
Processing Record 239 | half moon bay
Processing Record 240 | antsirabe
Processing Record 241 | bismarck
Processing Record 242 | lebu
Processing Record 243 | lisakovsk
Processing Record 244 | tutoia
Processing Record 245 | yar-sale
Processing Record 246 | thompson
Processing Record 247 | pemangkat
City not found. Skipping...
Processing Record 248 | ust-maya
City not found. Skipping...
Processing Record 249 | truro
Processing Record 250 | odweyne
Processing Record 251 | sitka
City not found. Skipping...
Processing Record 252 | qui nhon
Processing Record 253 | aklavik
Processing Record 254 | sao filipe
Processing Record 

Processing Record 452 | clyde river
Processing Record 453 | puerto escondido
Processing Record 454 | mount isa
Processing Record 455 | lugovoy
Processing Record 456 | biak
Processing Record 457 | marsh harbour
Processing Record 458 | aksarka
Processing Record 459 | beyneu
City not found. Skipping...
City not found. Skipping...
Processing Record 460 | qingyuan
Processing Record 461 | zyryanovsk
Processing Record 462 | port lincoln
Processing Record 463 | klaksvik
Processing Record 464 | hualmay
Processing Record 465 | te anau
Processing Record 466 | tekkali
Processing Record 467 | persianovskiy
Processing Record 468 | cayenne
Processing Record 469 | brae
Processing Record 470 | kisangani
Processing Record 471 | santiago del estero
Processing Record 472 | tautira
Processing Record 473 | severnoye
Processing Record 474 | naze
City not found. Skipping...
City not found. Skipping...
Processing Record 475 | asau
Processing Record 476 | keti bandar
Processing Record 477 | sainte-maxime
Proces

In [ ]:
# don't run this cell - sample export

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# don't rund this - sample

In [ ]:
# don't run this - sample

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
# don't run this -s ample

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# don't run this = sample

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# don't run this - sample

## Latitude vs. Humidity Plot

In [ ]:
# don't run this - sample

## Latitude vs. Cloudiness Plot

In [ ]:
# don't rum this - sample

## Latitude vs. Wind Speed Plot

In [ ]:
# don't rum this - sample

## Linear Regression

In [ ]:
# don't rum this - sample

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# don't rum this - sample

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# don't rum this - sample

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# don't rum this - sample

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# don't rum this - sample

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# don't rum this - sample

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# don't rum this - sample

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# don't rum this - sample

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# don't rum this - sample